In [85]:
import twitter
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import sys, os
from time import time, sleep, strftime
from datetime import timedelta
from IPython.display import clear_output

In [31]:
apps = [twitter.Api(consumer_key='Xn1blgiGIy6JDIMECCx9Q5xiS',
                      consumer_secret='xuZxo9o4M057DvNESJYTYSxIWjl9JXI10ReOxjFVtUBNNJ31AX',
                      access_token_key='864557009659731970-HqvZclihn4nXHmt7JVzKpFGnTNXn6w5',
                      access_token_secret='cndQ5gvAas2yxQNIIWLHIBBBMr8akrpi6vwwrZiWjdCGm'),
        twitter.Api(consumer_key='hCsMt4Nve0OwTprnnJr4D3vbe',
                      consumer_secret='bnnVZVPLKAT1ViRrYBbdhWmcvl7pzNMYWag7cBbEhCKhqiYRVj',
                      access_token_key='864557009659731970-nrZUBBDtSogU1dhpitGhukAfBFwikX7',
                      access_token_secret='DvVTpsGBMT5K0zlj1FgeAMusHnS17o8AqaPXUVccRA15u'),
        twitter.Api(consumer_key='dN0iRkbCwEWHn0eXgNOKlIw6M',
                      consumer_secret='LYQKnfyoB995aTso8pwR32ty8dyILqGkAshoqigirrqPbHz2uY',
                      access_token_key='864557009659731970-e1MuVE0XhhkkAYa86z6coEJ8IBqjo0i',
                      access_token_secret='6y1XWbjxpXXDFtTolmShVVPBIMM71pX7wkYSmsLVlRExM'),
        twitter.Api(consumer_key='QP6HEbTvAgEYpOHv1MeWs8H0W',
                      consumer_secret='857sE9cTgejErJEvXa0xiFOEDNXgYD37jVZ2PcHt32yftNeBRu',
                      access_token_key='864557009659731970-um3JJ4uuPUXpD3NkaYkOLOuYrj8wgA5',
                      access_token_secret='Sy4vyzGznc7Kh8XCdC2kwj0evM2j6mc3LgDGC1z7QxEV3')]

In [43]:
user_data = pd.read_csv('userdata.csv')['twitter'].tolist()
users = np.unique(['@%s' % user for user in user_data])
lang = ['en']

In [15]:
!! mkdir data
clear_output()

In [16]:
def msg(func):
    def new_func():
        start = time()
        func()
        end = time()
        print('ready for %s;   time: %s' % (*args[0], str(timedelta(seconds=end - start))))
    return new_func

In [79]:
def scratch_tweets(name, app):
    start = time()
    
    try:
        if os.path.exists('data/%s.csv' % name):
            return
        max_id = None
        sdf = np.array([])
        user = app.GetUser(screen_name=name)
        tail = user.statuses_count % 200
        bound = user.statuses_count // 200

        for i in range(bound + 1):
            if i < bound:
                slist = app.GetUserTimeline(screen_name=name, count=200, max_id=max_id)
            else:
                slist = app.GetUserTimeline(screen_name=name, count=tail, max_id=max_id)
            for j, s in enumerate(slist):
                if s.lang in lang:
                    text = s.text
                    tags = [h.text for h in s.hashtags]
                    sdf = np.append(sdf, {'text': text, 'tags': tags})
                    if j == len(slist) - 1:
                        max_id = s.id
        sdf = pd.DataFrame.from_records(sdf)
        sdf.columns = ['text', 'tags']
        sdf.to_csv('data/%s.csv' % name, index=False)
        end = time()
        print('ready for %s;   time: %s' % (name, str(timedelta(seconds=end - start))))
    except Exception as e:
        if isinstance(e, twitter.TwitterError):
            print(e.message)
            if e.message == 'Not authorized':
                return
            if isinstance(e.message[0], dict):
                if e.message[0]['code'] == 88:
                    print('Rate limit exceeded; sleeping')
                    sleep(1000)
        else:
            print(e, name)

In [80]:
def parallel_apps(i):
    app = apps[i]
    names = np.array_split(users, len(apps))[i]
    [scratch_tweets(name, app) for name in names]

In [ ]:
Parallel(n_jobs=-1)(delayed(parallel_apps)(i) for i in range(len(apps)))

Length mismatch: Expected axis has 0 elements, new values have 2 elements @YALINonline
Length mismatch: Expected axis has 0 elements, new values have 2 elements @YASSER_Q_Y20
Length mismatch: Expected axis has 0 elements, new values have 2 elements @LAAZCARATE
[{'message': 'User not found.', 'code': 50}]
Length mismatch: Expected axis has 0 elements, new values have 2 elements @AdnanAlarour
[{'message': 'User not found.', 'code': 50}]
Length mismatch: Expected axis has 0 elements, new values have 2 elements @YosriFouda
[{'message': 'User not found.', 'code': 50}]
Length mismatch: Expected axis has 0 elements, new values have 2 elements @MTLovenHoney
Length mismatch: Expected axis has 0 elements, new values have 2 elements @MustafaHosny
Length mismatch: Expected axis has 0 elements, new values have 2 elements @AlbertoCiurana
Not authorized.
Length mismatch: Expected axis has 0 elements, new values have 2 elements @ahmethc
Length mismatch: Expected axis has 0 elements, new values have 2 

Process ForkPoolWorker-266:
Process ForkPoolWorker-269:
Process ForkPoolWorker-270:
Process ForkPoolWorker-271:
Traceback (most recent call last):
Process ForkPoolWorker-274:
Process ForkPoolWorker-275:
Process ForkPoolWorker-273:
Traceback (most recent call last):
Process ForkPoolWorker-272:
Process ForkPoolWorker-276:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/daria/anaconda3/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/daria/anaconda3/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/daria/anaconda3/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
  File "/home/daria/anaconda3/lib/python3.5/multiprocessing/process.py", line 254, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/daria/

In [89]:
for user in np.unique(users):
    if os.path.exists('data/userdata/%s.csv' % user):
#         print(user)
        data = pd.read_csv('data/userdata/%s.csv' % user, encoding='utf8', engine='python')
        data = data.mask(data.applymap(str).eq('[]')).dropna()
        with open('filtered.csv', 'a') as dfile:
            data.to_csv(dfile, index=False)

In [90]:
data = pd.read_csv('filtered.csv')

In [91]:
data.shape

(1627633, 2)

In [92]:
data = data.drop_duplicates()

In [93]:
data.shape

(282382, 2)

In [94]:
data.to_csv('filtered.csv', index=False)

In [95]:
for user in np.unique(users):
    if os.path.exists('data/userdata/%s.csv' % user):
#         print(user)
        data = pd.read_csv('data/userdata/%s.csv' % user, encoding='utf8', engine='python')
        print(data.shape)

(1151, 2)
(118, 2)
(6188, 2)
(2965, 2)
(137, 2)
(326, 2)
(598, 2)
(17446, 2)
(3193, 2)
(24349, 2)
(175359, 2)
(109, 2)
(4595, 2)
(12, 2)
(29, 2)
(91, 2)
(2643, 2)
(516, 2)
(3383, 2)
(143, 2)
(846, 2)
(270, 2)
(26, 2)
(1785, 2)
(14417, 2)
(278, 2)
(25400, 2)
(130, 2)
(191, 2)
(26, 2)
(32213, 2)
(76, 2)
(922, 2)
(1232, 2)
(2706, 2)
(4055, 2)
(2118, 2)
(3040, 2)
(26932, 2)
(3036, 2)
(31316, 2)
(19733, 2)
(9332, 2)
(6138, 2)
(3162, 2)
(37423, 2)
(2860, 2)
(778, 2)
(3619, 2)
(5641, 2)
(843, 2)
(771, 2)
(27597, 2)
(2807, 2)
(363, 2)
(23075, 2)
(8, 2)
(18005, 2)
(2215, 2)
(2358, 2)
(1846, 2)
(19141, 2)
(163, 2)
(10218, 2)
(3120, 2)
(890, 2)
(3382, 2)
(567, 2)
(5437, 2)
(5772, 2)
(254, 2)
(348, 2)
(2917, 2)
(15, 2)
(3004, 2)
(635, 2)
(242, 2)
(4505, 2)
(6244, 2)
(5854, 2)
(182, 2)
(310, 2)
(6646, 2)
(3123, 2)
(3937, 2)
(4442, 2)
(61, 2)
(1476, 2)
(567, 2)
(11700, 2)
(1399, 2)
(26462, 2)
(2910, 2)
(2460, 2)
(2139, 2)
(32, 2)
(819, 2)
(3192, 2)
(19205, 2)
(3807, 2)
(3184, 2)
(714, 2)
(76, 2)
(27

KeyboardInterrupt: 